In [ ]:
import pathlib
import hashlib
import subprocess
import collections
import tqdm
import concurrent.futures


In [35]:
def py_md5(file_path):
    file_path = pathlib.Path(file_path)
    return hashlib.md5(file_path.read_bytes()).hexdigest()

In [37]:
def sh_md5(file_path):
    file_path = pathlib.Path(file_path)
    return subprocess.check_output(['md5sum', file_path]).decode().split()[0]

In [ ]:
path = pathlib.Path("/home/mila/g/gagnonju/scratch/lambdal_marglicot_openinstruct/open_instruct_output")
safetensors = list(path.glob("**/*.*"))

In [ ]:
indices_by_folder = collections.defaultdict(list)
by_folder = collections.defaultdict(list)

for safetensor in safetensors:
    run_folder_name = safetensor.parent.parent.relative_to(path)
    
    # Save the folder per run
    by_folder[run_folder_name].append(safetensor)
    
    # Save the index per run
    index = int(safetensor.parent.name.split("_")[1])
    indices_by_folder[run_folder_name].append(index)

indices_by_folder =  {k: sorted(v) for k, v in indices_by_folder.items()}
indices_by_folder

In [44]:
# Merkel tree by folder
merkel_tree = {}
total_str = ""

with concurrent.futures.ThreadPoolExecutor() as executor:
    for run_folder_name in sorted(by_folder):
        merkel_tree[run_folder_name] = []

        for safetensor in sorted(by_folder[run_folder_name]):
            merkel_tree[run_folder_name].append(executor.submit(sh_md5, safetensor))

    for run_folder_name in tqdm.tqdm(by_folder):
        print(run_folder_name)
        concatenated = "".join([future.result() for future in merkel_tree[run_folder_name]])
        total_str += hashlib.md5(concatenated.encode()).hexdigest()

hashlib.md5(total_str.encode()).hexdigest()

  0%|          | 0/3 [00:00<?, ?it/s]

2025-02-10_19-32-40_rlvr_gsm8k_only_smollm2_instruct_checkpoints


 33%|███▎      | 1/3 [01:12<02:25, 72.68s/it]

2025-02-10_19-33-46_rlvr_math_only_smollm2_instruct_checkpoints


 67%|██████▋   | 2/3 [02:10<01:03, 63.80s/it]

2025-02-10_19-35-25_rlvr_gsm8k_math_smollm2_instruct_checkpoints


100%|██████████| 3/3 [02:35<00:00, 51.96s/it]


'532c80a6b240066371eef7d982e6e84e'